In [8]:
import tweepy as tw
import pandas as pd
import re
import glob
import os
import schedule
import time
import requests
from textblob import TextBlob 
from datetime import datetime

%run ./fair_keys.ipynb

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

url = "https://api.aiforthai.in.th/ssense"
headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}

In [9]:
#keys = ['Spy x Family','Genshin Impact']
keys = ['bl anime','anime comedy','anime romance','ต่างโลก','anime','animation','shounen','pixar',
        'harem','fantasy anime','sport anime','attack on titan','disney animation','animation studio',
        'shounen ai','shoujo','อนิเมะ','2d animation','อนิเมะแนะนำ','japan animation']

In [10]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [11]:
def extract_hashtags(text):
    regex = "#(\w+)" 
    hashtag_list = re.findall(regex, text)
    return hashtag_list

In [12]:
def getSentiment(text):
    # set sentiment
    if TextBlob(text).sentiment.polarity > 0:
        return 'positive'
    elif TextBlob(text).sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [13]:
def savedata():
    def get_related_tweets(key_word):
        tweet_keyword = []
        twitter_users = []
        twitter_users_location = []
        tweet_hashtag = []
        tweet_time = []
        tweet_string = [] 
        tweet_countRT = []
        tweet_fav = []
        tweet_sentiment = []
        tweet_polarity = []
        tweet_language = []
        for tweet in tw.Cursor(api.search_tweets,
                                q=key_word,
                                tweet_mode="extended",
                                include_entities=True).items(100):
                                
            if(tweet.lang == 'en' or tweet.lang == 'th'):
                twitter_users.append(tweet.user.screen_name)#
                twitter_users_location.append(tweet.user.location)#
                tweet_time.append(tweet.created_at)#
                tweet_string.append(remove_url(tweet.full_text))#
                tweet_countRT.append(tweet.retweet_count)#
                tweet_fav.append(tweet.favorite_count)#
                tweet_keyword.append(key_word)#
                tweet_hashtag.append(str(extract_hashtags(tweet.full_text)))#
                tweet_language.append(tweet.lang)#
                if tweet.lang == 'en':
                    tweet_polarity.append(getSentiment(tweet.full_text))
                    tweet_sentiment.append(TextBlob(tweet.full_text).sentiment.polarity)
                elif tweet.lang == 'th':
                    text = re.sub(r'[%]',' ',tweet.full_text)
                    params = {'text':text}
                    response = requests.get(url, headers=headers, params=params)
                    #print(url,headers,params)
                    #print(response.json())
                    #print(str(response.json()['sentiment']['polarity']))
                    #print(str(response.json()['sentiment']['score']))
                    try:
                        polarity = str(response.json()['sentiment']['polarity'])
                        sentiment = str(response.json()['sentiment']['score'])
                    except (KeyError):
                        polarity = 'neutral'
                        sentiment = 0
                    tweet_polarity.append(polarity)
                    tweet_sentiment.append(sentiment)

        df = pd.DataFrame({'Keyword':tweet_keyword,'User':twitter_users,'Tweet': tweet_string,'Language':tweet_language, 'Time': tweet_time,'User Location':twitter_users_location,
                            'Hashtag':tweet_hashtag,'Polarity':tweet_polarity,'Likes':tweet_fav,'Retweet':tweet_countRT,'Sentiment':tweet_sentiment})
        #df.to_csv(f"{key_word}.csv")
        return df

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    today = datetime.today()
    filename = str("tweet_data_"+str(today.day)+str(today.month)+str(today.year)+".csv")

    if filename not in glob.glob("*.csv"):
        df = pd.DataFrame(columns=['Keyword','User','Tweet','Language','Time','User Location','Hashtag','Polarity','Likes','Retweet','Sentiment'])
    else:
        df = pd.read_csv(filename)
    
    for keyword in keys:
        df = pd.concat([df,get_related_tweets(keyword)])
    #print(df)
    #df['Time'] = pd.to_datetime(df['Time'])
    #df['Likes'] = df['Likes'].astype(str).astype(int)
    #df['Retweet'] = df['Retweet'].astype(str).astype(int)
    df.drop_duplicates(keep='last',inplace=True)
    df.sort_values(by=['Keyword'],inplace=True)
    if filename in glob.glob("*.csv"):
        os.remove(filename)
    df.to_csv(filename,encoding='utf-8',index=False)
    print('save complete',current_time)

In [14]:
savedata()
schedule.every(15).minutes.do(savedata)
while True:
    schedule.run_pending()
    time.sleep(1)

save complete 19:35:05
save complete 19:51:32
save complete 20:07:56
save complete 20:24:22
save complete 20:40:49
save complete 20:57:19


In [ ]:
# import requests
 
# url = "https://api.aiforthai.in.th/ssense"
 
# text = 'RT @manopsi: Russia-Ukraine war ทำให้ supply ข้าวสาลีโลกหายไป 25% ประเทศที่ได้รับผลกระทบมากล้วนเป็นประเทศยากจนในแอฟริกา'
 
# data = {'text':text}
 
# headers = {
#     'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"
#     }
 
# response = requests.post(url, data=data, headers=headers)
 
# response.json()

In [ ]:
# text = 'RT @manopsi: Russia-Ukraine war ทำให้ supply ข้าวสาลีโลกหายไป 25% ประเทศที่ได้รับผลกระทบมากล้วนเป็นประเทศยากจนในแอฟริกา'
# re.sub(r'[%]',' ',text)

In [ ]:
#df